In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

2023-03-11 00:10:12.123305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

in_directory = "/Users/jessemaccormac/Desktop/School/Fourth Year/ELEC 498/Data Recording/Frames"

data_count = {
    "LookLeft": 0,
    "LookRight": 0,
    "LookUp": 0,
    "LookDown": 0,
    "OpenMouth": 0,
    "RaiseEyebrows": 0,
    "Smile": 0,
}

for filename in os.listdir(in_directory):
    label, _ = filename.split("_",1)
    data_count[label] = data_count[label] + 1
    
print(data_count)
    

{'LookLeft': 151, 'LookRight': 106, 'LookUp': 112, 'LookDown': 108, 'OpenMouth': 181, 'RaiseEyebrows': 181, 'Smile': 254}


In [6]:
import pandas as pd

out_directory = "/Users/jessemaccormac/Desktop/School/Fourth Year/ELEC 498/Data Recording/Mesh_Images"
df = pd.DataFrame(columns=['Labels', 'Landmarks'])

# For static images:
# IMAGE_FILES = os.listdir(in_directory)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for file in os.listdir(in_directory):
    file_path = os.path.join(in_directory,file)
    image = cv2.imread(file_path)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    results.multi_face_landmarks

    label,_ = file.split("_",1)
    # df = df.append(['Label'=label,'Landmarks'=results.multi_face_landmarks])
    # new_row = {'Labels': label, 'Landmarks': results.multi_face_landmarks[0].landmark}
    df.loc[len(df)] = [label] + [[(lm.x, lm.y, lm.z) for lm in results.multi_face_landmarks[0].landmark]]
# results.multi_face_landmarks[0].landmark
# for face_landmarks in results.multi_face_landmarks:
#       print('face_landmarks:', face_landmarks)
df.head()
df['Labels'].value_counts()


df.to_csv("/Users/jessemaccormac/Desktop/School/Fourth Year/ELEC 498/Data Recording/landmark_data.csv", index=False)

In [ ]:
out_directory = "/Users/jessemaccormac/Desktop/School/Fourth Year/ELEC 498/Data Recording/Mesh_Images"

# For static images:
# IMAGE_FILES = os.listdir(in_directory)
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for file in os.listdir(in_directory):
    file_path = os.path.join(in_directory,file)
    image = cv2.imread(file_path)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      # print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
    filename, ext = os.path.splitext(file)
    file_out = os.path.join(out_directory, filename + '.jpg')   
    cv2.imwrite(file_out, annotated_image)

In [ ]:
xyz = [(lm.x, lm.y, lm.z) for lm in results.multi_face_landmarks[0].landmark]
# landmarks = pd.Series(results.multi_face_landmarks[0].landmark)
print(len(xyz))

df['Labels'].value_counts()

In [ ]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()